## importing chromadb library 
import chromadb

In [2]:
import chromadb
import uuid

client = chromadb.Client()

## creating collection 

In [3]:
collection_name = "policies"

# Check if a collection with this name already exists
""" existing_collections = client.list_collections()
existing_names = [c.name for c in existing_collections]

if collection_name in existing_names:
    collection = client.get_collection(name=collection_name)
    print(f"Collection '{collection_name}' already exists. Using existing collection.")
else:
    collection = client.create_collection(name=collection_name)
    print(f"Collection '{collection_name}' created successfully!") """

collection = client.get_or_create_collection(name=collection_name)

In [4]:
from sentence_transformers import SentenceTransformer

# 1. Load the full policies text from file
with open("policies.txt", "r", encoding="utf-8") as f:
    policy_text = f.read()

# 2. Split into paragraph-level chunks (separated by blank lines)
raw_chunks = [p.strip() for p in policy_text.split("\n\n") if p.strip()]
print(f"Total paragraphs (chunks) found: {len(raw_chunks)}")

# 3. Initialize the embedding model (once per notebook)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 4. Generate UUID ids for each chunk
ids = [str(uuid.uuid4()) for _ in raw_chunks]

# 5. Embed each chunk
embeddings = model.encode(raw_chunks).tolist()  # list of [dim]-vectors

# 6. Add all chunks to the 'policies' collection
collection.add(
    ids=ids,
    documents=raw_chunks,
    embeddings=embeddings,
    metadatas=[{"source": "policies.txt", "type": "paragraph"} for _ in raw_chunks],
)

print("Policy paragraphs added to Chroma collection.")

d:\2025\VDB\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total paragraphs (chunks) found: 33
Policy paragraphs added to Chroma collection.


In [5]:
# Query the policies collection with a natural language question
query_text = "who is this meant for"

# Embed the query
query_embedding = model.encode([query_text]).tolist()

# Retrieve the top-k most similar paragraphs from Chroma
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
)

# Pretty-print the results
for i, (doc, meta, dist) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
    print(f"Result {i+1}")
    print("Distance:", dist)
    print("Metadata:", meta)
    print("Document:\n", doc)
    print("-" * 80)

Result 1
Distance: 1.5577661991119385
Metadata: {'source': 'policies.txt', 'type': 'paragraph'}
Document:
 Accessibility:
- The Store aims to provide an accessible environment for all customers, including individuals with disabilities.
- Requests for reasonable accommodations can be made to store management or customer service staff.
--------------------------------------------------------------------------------
Result 2
Distance: 1.7276356220245361
Metadata: {'type': 'paragraph', 'source': 'policies.txt'}
Document:
 By shopping at BrightCart Retail or using our services, you acknowledge that you have read, understood, and agree to abide by these policies, as may be amended from time to time.
--------------------------------------------------------------------------------
Result 3
Distance: 1.7711091041564941
Metadata: {'source': 'policies.txt', 'type': 'paragraph'}
Document:
 9. Health, Safety, and Accessibility
Health and Safety:
- Customers are required to comply with all posted he

In [6]:
# Configure Gemini LLM using API key from .env
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not set in .env. Please edit .env and set GEMINI_API_KEY=your_actual_key.")

# Configure Gemini client
genai.configure(api_key=api_key)

# Use the specified Gemini model
llm = genai.GenerativeModel("gemini-2.5-flash")
print("Gemini LLM (gemini-2.5-flash) initialized.")

Gemini LLM (gemini-2.5-flash) initialized.


In [7]:
# RAG-style helper: retrieve from Chroma, then answer with Gemini

def answer_policies_question(question: str, k: int = 3) -> str:
    """Retrieve top-k policy paragraphs relevant to the question and ask Gemini to answer.

    Assumes:
    - `collection` is a Chroma collection with policy paragraphs.
    - `model` is the SentenceTransformer embedding model.
    - `llm` is the configured Gemini GenerativeModel.
    """
    # 1. Embed the question
    query_embedding = model.encode([question]).tolist()

    # 2. Retrieve top-k paragraphs from Chroma
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k,
    )

    retrieved_docs = results["documents"][0]

    # 3. Build a context string from the retrieved paragraphs
    context = "\n\n".join(retrieved_docs)

    # 4. Construct a prompt for Gemini
    prompt = f"""You are a helpful assistant for a retail store.
Use ONLY the information in the CONTEXT section below to answer the QUESTION.
If the answer is not clearly present in the context, say that you don't know based on the given policies.

CONTEXT:
{context}

QUESTION:
{question}

Answer clearly and concisely.
"""

    response = llm.generate_content(prompt)

    # `response.text` is usually the main text answer. Fallback to str(response) if needed.
    return getattr(response, "text", str(response))


# Example usage (you can change the question text and re-run):
example_answer = answer_policies_question("What is the return policy on unopened items?", k=3)
print(example_answer)

Most unopened items in new and resalable condition may be returned within 30 days of purchase with an original receipt.
